# **Data Wrangling Lab**


In this assignment you will be performing data wrangling.


## Objectives


-   Identify duplicate values in the dataset.

-   Remove duplicate values from the dataset.

-   Identify missing values in the dataset.

-   Impute the missing values in the dataset.

-   Normalize data in the dataset.


<hr>


Import pandas module.


In [7]:
import pandas as pd

<ipython-input-7-7dd3504c366f>:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Load the dataset into a dataframe.


<h2>Read Data</h2>
<p>
We utilize the <code>pandas.read_csv()</code> function for reading CSV files. However, in this version of the lab, which operates on JupyterLite, the dataset needs to be downloaded to the interface using the provided code below.
</p>


The functions below will download the dataset into your browser:


In [8]:
from pyodide.http import pyfetch

async def download(url, filename):
    response = await pyfetch(url)
    if response.status == 200:
        with open(filename, "wb") as f:
            f.write(await response.bytes())

In [9]:
file_path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/LargeData/m1_survey_data.csv"

To obtain the dataset, utilize the download() function as defined above:  


In [10]:
await download(file_path, "m1_survey_data.csv")
file_name="m1_survey_data.csv"

Utilize the Pandas method read_csv() to load the data into a dataframe.


In [11]:
df = pd.read_csv(file_name)

> Note: This version of the lab is working on JupyterLite, which requires the dataset to be downloaded to the interface.While working on the downloaded version of this notebook on their local machines(Jupyter Anaconda), the learners can simply **skip the steps above,** and simply use the URL directly in the `pandas.read_csv()` function. You can uncomment and run the statements in the cell below.


In [12]:
#df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/LargeData/m1_survey_data.csv")

## Finding duplicates


In this section you will identify duplicate values in the dataset.


 Find how many duplicate rows exist in the dataframe.


In [24]:
# your code goes here
duplicates = df[df.duplicated()]
print(f"Number of Duplicates found are : {duplicates.shape[0]}")
print(f"Sample Duplicates are :")
print(duplicates.head(5))

Number of Duplicates found are : 154
Sample Duplicates are :
      Respondent                      MainBranch Hobbyist  \
1168        2339  I am a developer by profession      Yes   
1169        2342  I am a developer by profession      Yes   
1170        2343  I am a developer by profession      Yes   
1171        2344  I am a developer by profession      Yes   
1172        2347  I am a developer by profession      Yes   

                                            OpenSourcer  \
1168                         Once a month or more often   
1169                                              Never   
1170  Less than once a month but more than once per ...   
1171                                              Never   
1172                                              Never   

                                             OpenSource          Employment  \
1168  OSS is, on average, of HIGHER quality than pro...  Employed full-time   
1169  The quality of OSS and closed source software ...  Em

## Removing duplicates


Remove the duplicate rows from the dataframe.


In [17]:
# your code goes here
df_cleaned = df.drop_duplicates()

Verify if duplicates were actually dropped.


In [18]:
# your code goes here
print(f"Duplicates removed : {df.shape[0]-df_cleaned.shape[0]}")

Duplicates removed : 154


## Finding Missing values


Find the missing values for all columns.


In [27]:
# your code goes here
missing_values = df_cleaned.isna().sum()
print(f"Missing Values :")
print(missing_values)

Missing Values :
Respondent        0
MainBranch        0
Hobbyist          0
OpenSourcer       0
OpenSource       81
               ... 
Sexuality       542
Ethnicity       675
Dependents      140
SurveyLength     19
SurveyEase       14
Length: 85, dtype: int64


Find out how many rows are missing in the column 'WorkLoc'


In [28]:
# your code goes here
missing_WorkLoc = df_cleaned['WorkLoc'].isna().sum()
print(f"Missing workLoc : ")
print(missing_WorkLoc)

Missing workLoc : 
32


## Imputing missing values


Find the  value counts for the column WorkLoc.


In [29]:
# your code goes here
value_counts = df_cleaned['WorkLoc'].value_counts()
print("\nValue counts for 'WorkLoc':")
print(value_counts)


Value counts for 'WorkLoc':
WorkLoc
Office                                            6806
Home                                              3589
Other place, such as a coworking space or cafe     971
Name: count, dtype: int64


Identify the value that is most frequent (majority) in the WorkLoc column.


In [31]:
#make a note of the majority value here, for future reference
#
most_frequent_value = value_counts.idxmax()
print(f"\nMost frequent value in 'WorkLoc': {most_frequent_value}")


Most frequent value in 'WorkLoc': Office


Impute (replace) all the empty rows in the column WorkLoc with the value that you have identified as majority.


In [34]:
# your code goes here
df['WorkLoc'].fillna(most_frequent_value, inplace = True) 

After imputation there should ideally not be any empty rows in the WorkLoc column.


Verify if imputing was successful.


In [35]:
# your code goes here
missing_workloc_after_imputation = df_cleaned['WorkLoc'].isna().sum()
print(f"\nNumber of missing values in 'WorkLoc' after imputation: {missing_workloc_after_imputation}")


Number of missing values in 'WorkLoc' after imputation: 32


## Normalizing data


There are two columns in the dataset that talk about compensation.

One is "CompFreq". This column shows how often a developer is paid (Yearly, Monthly, Weekly).

The other is "CompTotal". This column talks about how much the developer is paid per Year, Month, or Week depending upon his/her "CompFreq". 

This makes it difficult to compare the total compensation of the developers.

In this section you will create a new column called 'NormalizedAnnualCompensation' which contains the 'Annual Compensation' irrespective of the 'CompFreq'.

Once this column is ready, it makes comparison of salaries easy.


<hr>


List out the various categories in the column 'CompFreq'


In [36]:
# your code goes here
comp_freq_categories = df_cleaned['CompFreq'].unique()
print("Categories in 'CompFreq':")
print(comp_freq_categories)


Categories in 'CompFreq':
['Yearly' 'Monthly' 'Weekly' nan]


Create a new column named 'NormalizedAnnualCompensation'. Use the hint given below if needed.


Double click to see the **Hint**.

<!--

Use the below logic to arrive at the values for the column NormalizedAnnualCompensation.

If the CompFreq is Yearly then use the exising value in CompTotal
If the CompFreq is Monthly then multiply the value in CompTotal with 12 (months in an year)
If the CompFreq is Weekly then multiply the value in CompTotal with 52 (weeks in an year)

-->


In [ ]:
# your code goes here
def normalize_annual_compensation(row):
    if row['CompFreq'] == 'Yearly':
        return row['CompTotal']
    elif row['CompFreq'] == 'Monthly':
        return row['CompTotal'] * 12
    elif row['CompFreq'] == 'Weekly':
        return row['CompTotal'] * 52
    else:
        return None
df_cleaned['NormalizedAnnualCompensation'] = df_cleaned.apply(normalize_annual_compensation, axis=1)


<!--## Change Log


<!--| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-10-17        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |--!>
